In [6]:
import pandas as pd
import numpy as np
import requests
from Bio import Entrez

import scanpy as sc

/home/icb/dominik.klein/mambaforge/envs/esm_cfp/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/icb/dominik.klein/mambaforge/envs/esm_cfp/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/icb/dominik.klein/mambaforge/envs/esm_cfp/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/icb/dominik.klein/mambaforge/envs/esm_cfp/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/icb/dominik.klein/mambaforge/envs/esm_cfp/lib/py

In [7]:
adata = sc.read_h5ad("/lustre/groups/ml01/workspace/ten_million/data/final_data/outputDIR_LV/adata_hvg2000_LV.h5ad")

In [8]:
cytokines = adata.obs["cytokine"].unique()

In [82]:
def fetch_protein_sequence(protein_name: str, return_fasta=False) -> str:
    Entrez.email = "your_email@example.com"  # Replace with your email
    # Search for the protein in the NCBI database
    handle = Entrez.esearch(db="protein", term=protein_name)
    record = Entrez.read(handle)
    handle.close()
    if record['IdList']:
        protein_id = record['IdList'][0]
        # Fetch the protein sequence
        handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
        fasta_record = handle.read()
        handle.close()
        if return_fasta:
            return fasta_record.strip()  # Return the full FASTA format
        else:
            # Extract the amino acid sequence
            sequence_lines = fasta_record.strip().split("\n")[1:]  # Skip the first line (header)
            amino_sequence = ''.join(sequence_lines)  # Join the lines to get the full sequence
            return amino_sequence
    else:
        return "Protein not found."

In [86]:
with open("cytokines.fasta", "w") as fasta_file:
    for cytokine in cytokines:
        protein_sequence = fetch_protein_sequence(cytokine)
        fasta_file.write(f">{cytokine}\n")
        fasta_file.write(f"{protein_sequence}\n")